# Installing Required Libraries


In [ ]:

!pip install pytubefix
!pip install opencv-python
!pip install ultralytics numpy filterpy deep-sort-realtime
!pip install inference_sdk
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 10.2 MB/s eta 0:00:00


# Getting the dataset from Roboflow to train out model
(can make our own dataset by labelling images and getting it ready to train YOLO models with the help of Roboflow)


In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="Get your api key from roboflow website")
project = rf.workspace("roboflow-universe-projects").project("basketball-players-fy4c2")
version = project.version(23)
dataset = version.download("yolov8")

# Might need to change the path of train, test and val datasets in data.yaml file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Basketball-Players-23 in yolov8:: 100%|██████████| 2404/2404 [00:00<00:00, 3135.57it/s]


#Train the model

In [ ]:
!yolo task=detect mode=train model=yolov8m.yaml data={dataset.location}/data.yaml epochs=20 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/content/Basketball-Players-23/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, i

# Validate the model


In [ ]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=640

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 92 layers, 25,844,971 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/Basketball-Players-23/valid/labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100% 32/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.32it/s]
                   all         32        477      0.796      0.712       0.74      0.397
                  Ball         18         18          1          0     0.0118     0.0094
                  Hoop         22         22      0.786      0.835      0.876      0.511
                Period         22         23      0.643      0.565      0.673      0.329
                Player         31        253      0.854      0.791      0.877      0.465
                   Ref         28         55      0.839      0.836      0.827      0.488
            Shot Clock         19         19

# Testing our model on test dataset


In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=640

WARNING ⚠️ 'source' argument is missing. Using default 'source=/usr/local/lib/python3.11/dist-packages/ultralytics/assets'.
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 92 layers, 25,844,971 parameters, 0 gradients, 78.7 GFLOPs

image 1/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 Players, 55.9ms
image 2/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/zidane.jpg: 384x640 1 Player, 42.3ms
Speed: 3.2ms preprocess, 49.1ms inference, 94.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


#Displaying the tested images


In [ ]:
import glob
from IPython.display import Image,display

for image_path in glob.glob(f"/content/runs/detect/predict/*.jpg"):
  display(Image(filename=image_path, height=600))
  print("\n")

In [ ]:
import cv2
from ultralytics import YOLO
from pytubefix import YouTube
from google.colab.patches import cv2_imshow
import time
from google.colab import files  # Added this import

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")


#Testing our model on a video from youtube

In [ ]:
# Load trained YOLO model
model = YOLO("runs/detect/train/weights/best.pt")
url = "https://www.youtube.com/watch?v=A9kqcnP6Hyo"
yt = YouTube(url)
stream = yt.streams.filter(progressive=True, file_extension="mp4").first()
if not stream:
    raise Exception("No suitable MP4 stream found.")
stream_url = stream.url

# Open the video stream
cap = cv2.VideoCapture(stream_url)
if not cap.isOpened():
    raise Exception("Error: Could not open video stream")

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# Process video frames
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Stream ended or failed to grab frame")
        break

    # Make predictions using YOLO model
    results = model.predict(frame)
    annotated_frame = results[0].plot()

    # Write the frame to the output video
    out.write(annotated_frame)

    frame_count += 1
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames")

# Release resources
cap.release()
out.release()
print("Processing complete. Video saved as 'output.mp4'")

# Download the output video
files.download('output.mp4')

Streaming output truncated to the last 5000 lines.
0: 384x640 1 Hoop, 6 Players, 1 Team Name, 3 Team Pointss, 1 Time Remaining, 12.2ms
Speed: 1.6ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 Players, 1 Team Name, 3 Team Pointss, 1 Time Remaining, 11.8ms
Speed: 1.6ms preprocess, 11.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Hoop, 7 Players, 2 Team Pointss, 1 Time Remaining, 14.3ms
Speed: 1.4ms preprocess, 14.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Players, 2 Team Pointss, 2 Time Remainings, 19.0ms
Speed: 2.3ms preprocess, 19.0ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 Players, 3 Team Pointss, 2 Time Remainings, 26.2ms
Speed: 2.3ms preprocess, 26.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 Players, 1 Team Name, 3 Team Pointss, 2 Time Remainings, 13.0ms
Speed: 1.7ms preprocess

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>